In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Carga y limpieza de datos

## (Hecho con la info obtenida en el notebook LlenadoDeNulls)

In [2]:
test =  pd.read_csv('data/test_full.csv', parse_dates = ['fecha'])
train = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])

In [3]:
train['habitaciones']=train['habitaciones'].astype(np.int8)
test['habitaciones']=test['habitaciones'].astype(np.int8)

train['metroscubiertos']=train['metroscubiertos'].astype(np.int16)
test['metroscubiertos']=test['metroscubiertos'].astype(np.int16)

train['metrostotales']=train['metrostotales'].astype(np.int16)
test['metrostotales']=test['metrostotales'].astype(np.int16)

train['antiguedad']=train['antiguedad'].astype(np.uint8)
test['antiguedad']=test['antiguedad'].astype(np.uint8)

train['garages']=train['garages'].astype(np.int8)
test['garages']=test['garages'].astype(np.int8)

train['banos']=train['banos'].astype(np.int8)
test['banos']=test['banos'].astype(np.int8)

train['gimnasio']=train['gimnasio'].astype(np.int8)
train['usosmultiples']=train['usosmultiples'].astype(np.int8)
train['piscina']=train['piscina'].astype(np.int8)
train['escuelascercanas']=train['escuelascercanas'].astype(np.int8)
train['centroscomercialescercanos']=train['centroscomercialescercanos'].astype(np.int8)
test['gimnasio']=test['gimnasio'].astype(np.int8)
test['usosmultiples']=test['usosmultiples'].astype(np.int8)
test['piscina']=test['piscina'].astype(np.int8)
test['escuelascercanas']=test['escuelascercanas'].astype(np.int8)
test['centroscomercialescercanos']=test['centroscomercialescercanos'].astype(np.int8)

In [4]:
#Algunas columnas tienen nulls que no se como encararlos. Por ahora los relleno con 0
train=train.fillna(0)
test=test.fillna(0)

# KNN

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
import random

In [6]:
from sklearn import preprocessing 
def normalizar_data(df):
    min_max_scaler = preprocessing.MinMaxScaler() 
    np_scaled = min_max_scaler.fit_transform(df)
    return pd.DataFrame(np_scaled)

In [7]:
#Califico las fechas de acuerdo a la diferencia entre la fecha actual y la fecha mínima sobre el máximo de dias
fecha_min = train.fecha.min()
fecha_max = train.fecha.max()
delta_max_days = (fecha_max - fecha_min).days
def calificar_fecha(fecha):
    delta = fecha - fecha_min
    return delta.days/delta_max_days

In [3]:
knn = KNeighborsRegressor(n_neighbors=7, metric="manhattan", weights="distance", algorithm="brute")

In [9]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [22]:
#Count encoder para tipo de propiedad
count_tipo_prop = train.groupby("tipodepropiedad").count().reset_index()
train["tipo_prop_enc"] = train["tipodepropiedad"].map(lambda x: count_tipo_prop[count_tipo_prop["tipodepropiedad"]==x].iloc[0]["id"])

In [22]:
train["tipo_prop_enc"].to_frame().to_csv("data/tipo_prop_enc.csv", index=False)

In [11]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [12]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [46]:
train = normalizar_data(train)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int8, uint8, int16, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [13]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train.shape[0])
    knn.fit(train.drop(i), precio.drop(i))
    id_predic.append(i)
    predic.append(knn.predict(train.loc[[i]]))

In [14]:
real = []
for x in id_predic:
    real.append(precio[x])

In [15]:
mean_absolute_error(real, predic)

899323.1973590653

In [48]:
test["puntaje_por_fecha"] = test['fecha'].transform(lambda x: calificar_fecha(x))
test.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,puntaje_por_fecha
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29,3,2,...,0.0,19.408668,-99.246767,2013-07-20,0,0,0,0,0,0.309967


In [49]:
#Count encoding para tipo de propiedad
test["tipo_prop_enc"] = test["tipodepropiedad"].map(lambda x: count_tipo_prop[count_tipo_prop["tipodepropiedad"]==x].iloc[0]["id"]) 

In [50]:
test_id = test["id"]

In [51]:
test = test.drop(labels=col_drop, axis=1)

In [52]:
test = normalizar_data(test)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int8, uint8, int16, int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [53]:
knn.fit(train, precio)

KNeighborsRegressor(algorithm='brute', leaf_size=30, metric='manhattan',
          metric_params=None, n_jobs=None, n_neighbors=7, p=2,
          weights='distance')

In [54]:
res = knn.predict(test)

In [55]:
with open("data/knn_full.csv","w") as f:
    f.write("id,target\n")
    for i,v in enumerate(res):
        f.write("{},{}\n".format(test_id[i],v))

### Ahora predigo usando solo metroscubiertos, metrostotales, tipodepropiedad y puntaje_por_fecha

In [18]:
train = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])
train = train.fillna(0)

In [19]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [10]:
#Count encoder para tipo de propiedad
train["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc.csv")["tipo_prop_enc"]

In [23]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [24]:
col_drop_2 = ["antiguedad", "habitaciones", "garages", "banos", "gimnasio", "usosmultiples", "piscina", "escuelascercanas", "centroscomercialescercanos"]
train = train.drop(labels=col_drop_2, axis=1)

In [25]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [26]:
train = normalizar_data(train)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [27]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train.shape[0])
    knn.fit(train.drop(i), precio.drop(i))
    id_predic.append(i)
    predic.append(knn.predict(train.loc[[i]]))

In [28]:
real = []
for x in id_predic:
    real.append(precio[x])
    
mean_absolute_error(real, predic)

960896.831797947

### Ahora solo uso para clasificar las propiedades anteriores en fecha

In [29]:
train = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])
train = train.fillna(0)

In [30]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [32]:
#Count encoder para tipo de propiedad
train["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc.csv")["tipo_prop_enc"]

In [34]:
train = train.sort_values("fecha")

In [36]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [37]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [38]:
train = normalizar_data(train)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [39]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train.shape[0])
    knn.fit(train.drop([j for j in range(i,train.shape[0])]), precio.drop([j for j in range(i,train.shape[0])]))
    id_predic.append(i)
    predic.append(knn.predict(train.loc[[i]]))

In [40]:
real = []
for x in id_predic:
    real.append(precio[x])
    
mean_absolute_error(real, predic)

1818097.3617431908

### Ahora solo uso las propiedades que disten a lo sumo dos meses

In [109]:
train = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])
train = train.fillna(0)

In [110]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [78]:
#Count encoder para tipo de propiedad
train["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc.csv")["tipo_prop_enc"]

In [112]:
train = train.sort_values("fecha")

In [113]:
fechas = train["fecha"]

In [114]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [115]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [116]:
train = normalizar_data(train)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [156]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train.shape[0])
    rango = fechas.loc[lambda x: (fechas[i]-timedelta(180,0,0) < x) & (x < fechas[i]+timedelta(180,0,0))].index
    knn.fit(train.loc[rango].drop(i), precio.loc[rango].drop(i))
    id_predic.append(i)
    predic.append(knn.predict(train.loc[[i]]))

In [157]:
real = []
for x in id_predic:
    real.append(precio[x])
    
mean_absolute_error(real, predic)

1681721.5862329677

#### Ahora lo hago para el test

In [120]:
test =  pd.read_csv('data/test_full.csv', parse_dates = ['fecha'])
test = test.fillna(0)

In [121]:
#Set de test
test["puntaje_por_fecha"] = test['fecha'].transform(lambda x: calificar_fecha(x))
test.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,puntaje_por_fecha
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29,3,2,...,0.0,19.408668,-99.246767,2013-07-20,0,0,0,0,0,0.309967


In [122]:
#Count encoding para tipo de propiedad
test["tipo_prop_enc"] = test["tipodepropiedad"].map(lambda x: count_tipo_prop[count_tipo_prop["tipodepropiedad"]==x].iloc[0]["id"]) 

In [ ]:
test = test.sort_values("fecha")

In [123]:
fechas_test = test["fecha"]

In [124]:
test_id = test["id"]

In [125]:
test = test.drop(labels=col_drop, axis=1)

In [126]:
test = normalizar_data(test)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [127]:
res = knn.predict(test)

In [150]:
id_predic = []
predic = []

for fila in test.index:
    rango = fechas.loc[lambda x: (fechas_test[fila]-timedelta(60,0,0) < x) & (x < fechas_test[fila]+timedelta(60,0,0))].index
    knn.fit(train.loc[rango], precio.loc[rango])
    id_predic.append(i)
    predic.append(knn.predict(test.loc[[fila]]))

In [152]:
red = []
for x in predic:
    red.append(round(x[0]/1000)*1000)

In [153]:
with open("data/knn_dos_meses.csv","w") as f:
    f.write("id,target\n")
    for i,v in enumerate(red):
        f.write("{},{}\n".format(test_id[i],v))

### Uso solamente las columnas sin nulls

In [25]:
train = pd.read_csv('data/train.csv',parse_dates = ['fecha'])

In [12]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,NaN,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0,0.728368


In [13]:
#Count encoder para tipo de propiedad
count_tipo_prop = train.groupby("tipodepropiedad").count().reset_index()
train["tipo_prop_enc"] = train["tipodepropiedad"].map(lambda x: count_tipo_prop[count_tipo_prop["tipodepropiedad"]==x].iloc[0]["id"] if count_tipo_prop[count_tipo_prop["tipodepropiedad"]==x].shape[0]>0 else 0)

In [ ]:
train["tipo_prop_enc"].to_frame().to_csv("data/tipo_prop_enc_sin_nulls.csv", index=False)

In [14]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [15]:
train = train.dropna()

In [16]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [17]:
train = normalizar_data(train)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [18]:
precio = precio.reset_index()

In [19]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train.shape[0])
    knn.fit(train.drop(i), precio["precio"].drop(i))
    id_predic.append(i)
    predic.append(knn.predict(train.loc[[i]]))

In [20]:
real = []
for x in id_predic:
    real.append(precio.loc[x]["precio"])
    
mean_absolute_error(real, predic)

648382.7121751433

Ahora veo el score para las propiedades que tire del train

In [7]:
train_test = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])
train_test = train_test.fillna(0)

In [8]:
train_test["puntaje_por_fecha"] = train_test['fecha'].transform(lambda x: calificar_fecha(x))
train_test.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [9]:
#Count encoder para tipo de propiedad
train_test["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc.csv")["tipo_prop_enc"]

In [10]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train_test = train_test.drop(labels=col_drop, axis=1)

In [11]:
precio_tt = train_test["precio"]
train_test = train_test.drop(labels="precio", axis=1)

In [4]:
train = pd.read_csv('data/train.csv',parse_dates = ['fecha'])

In [12]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,NaN,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0,0.728368


In [13]:
#Count encoder para tipo de propiedad
train["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc_sin_nulls.csv")["tipo_prop_enc"]

In [14]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [15]:
train = train.dropna()

In [17]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [18]:
train_test = train_test.drop(train.index)
precio_tt = precio_tt.drop(train.index)
precio_tt = precio_tt.reset_index()

In [19]:
train_test = normalizar_data(train_test)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [20]:
train = normalizar_data(train)

In [21]:
precio = precio.reset_index()

In [22]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train_test.shape[0])
    knn.fit(train, precio["precio"])
    id_predic.append(i)
    predic.append(knn.predict(train_test.loc[[i]]))

In [23]:
real = []
for x in id_predic:
    real.append(precio_tt.loc[x]["precio"])
    
mean_absolute_error(real, predic)

1131366.3975394792

Ahora hago lo mismo pero con menos columnas

In [24]:
train_test = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])
train_test = train_test.fillna(0)

In [25]:
train_test["puntaje_por_fecha"] = train_test['fecha'].transform(lambda x: calificar_fecha(x))
train_test.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [26]:
#Count encoder para tipo de propiedad
train_test["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc.csv")["tipo_prop_enc"]

In [27]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
col_drop_2 = ["antiguedad", "garages", "banos", "habitaciones"]
train_test = train_test.drop(labels=col_drop+col_drop_2, axis=1)

In [28]:
precio_tt = train_test["precio"]
train_test = train_test.drop(labels="precio", axis=1)

In [29]:
train = pd.read_csv('data/train.csv',parse_dates = ['fecha'])

In [30]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,NaN,NaN,2015-08-23,0.0,0.0,0.0,0.0,0.0,2273000.0,0.728368


In [31]:
#Count encoder para tipo de propiedad
train["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc_sin_nulls.csv")["tipo_prop_enc"]

In [32]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
col_drop_2 = ["antiguedad", "garages", "banos", "habitaciones"]
train = train.drop(labels=col_drop+col_drop_2, axis=1)

In [33]:
train = train.dropna()

In [35]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [36]:
train_test = train_test.drop(train.index)
precio_tt = precio_tt.drop(train.index)
precio_tt = precio_tt.reset_index()

In [37]:
train_test = normalizar_data(train_test)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [38]:
train = normalizar_data(train)

In [39]:
precio = precio.reset_index()

In [40]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train_test.shape[0])
    knn.fit(train, precio["precio"])
    id_predic.append(i)
    predic.append(knn.predict(train_test.loc[[i]]))

In [41]:
real = []
for x in id_predic:
    real.append(precio_tt.loc[x]["precio"])
    
mean_absolute_error(real, predic)

1153028.0325529815

### Creo clusters por fecha

In [9]:
from sklearn.neighbors import NearestCentroid

In [21]:
train = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])

In [22]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [23]:
#Count encoder para tipo de propiedad
train["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc.csv")["tipo_prop_enc"]

In [24]:
train = train.sort_values("fecha")

In [25]:
clase_fecha = train.index.map(lambda x: x//500).to_frame()[0]

In [26]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [27]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [28]:
train = normalizar_data(train)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [29]:
nc = NearestCentroid(metric="manhattan")
nc.fit(train, clase_fecha)

NearestCentroid(metric='manhattan', shrink_threshold=None)

In [30]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train.shape[0])
    clase_pred = nc.predict(train.loc[[i]])
    rango = clase_fecha.loc[lambda x: x==clase_pred[0]].index
    knn.fit(train.loc[rango], precio.loc[rango])
    id_predic.append(i)
    predic.append(knn.predict(train.loc[[i]]))

In [31]:
real = []
for x in id_predic:
    real.append(precio[x])
    
mean_absolute_error(real, predic)

1828497.4802

### Creo clusters por rango de precios

In [67]:
train = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])

In [68]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [69]:
#Count encoder para tipo de propiedad
train["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc.csv")["tipo_prop_enc"]

In [70]:
train = train.sort_values("precio")

In [71]:
clase_precio = train.index.map(lambda x: x//500).to_frame()[0]

In [20]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [73]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [74]:
train = normalizar_data(train)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [75]:
nc = NearestCentroid(metric="manhattan")
nc.fit(train, clase_precio)

NearestCentroid(metric='manhattan', shrink_threshold=None)

In [76]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train.shape[0])
    clase_pred = nc.predict(train.loc[[i]])
    rango = clase_precio.loc[lambda x: x==clase_pred[0]].index
    knn.fit(train.loc[rango], precio.loc[rango])
    id_predic.append(i)
    predic.append(knn.predict(train.loc[[i]]))

In [77]:
real = []
for x in id_predic:
    real.append(precio[x])
    
mean_absolute_error(real, predic)

1775586.6384

### Ahora pruebo seleccionando rangos de las variables

In [159]:
train = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])

In [160]:
train["puntaje_por_fecha"] = train['fecha'].transform(lambda x: calificar_fecha(x))
train.head(1)

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,puntaje_por_fecha
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8,2,1,...,0.0,0.0,2015-08-23,0,0,0,0,0,2273000,0.728368


In [161]:
#Count encoder para tipo de propiedad
train["tipo_prop_enc"] = pd.read_csv("data/tipo_prop_enc.csv")["tipo_prop_enc"]

In [162]:
col_drop = ["lat","lng","ciudad","provincia","idzona","titulo","descripcion","id","direccion","tipodepropiedad","fecha"]
train = train.drop(labels=col_drop, axis=1)

In [163]:
precio = train["precio"]
train = train.drop(labels="precio", axis=1)

In [164]:
train_ref = train

In [165]:
train = normalizar_data(train)

C:\Users\Fernando\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [172]:
id_predic = []
predic = []

for x in range(5000):
    i = random.randrange(0, train.shape[0])
    rango = train_ref.loc[(train_ref.loc[i]["metroscubiertos"]-25<train_ref["metroscubiertos"]) & (train_ref["metroscubiertos"]<train_ref.loc[i]["metroscubiertos"]+25)].index
    #rango = train_ref.loc[rango].loc[(train_ref.loc[i]["metrostotales"]-25<train_ref["metrostotales"]) & (train_ref["metrostotales"]<train_ref.loc[i]["metrostotales"]+25)].index
    #rango = train_ref.loc[rango].loc[(train_ref.loc[i]["banos"]-1<train_ref["banos"]) & (train_ref["banos"]<train_ref.loc[i]["banos"]+1)].index
    rango = train_ref.loc[rango].loc[(train_ref.loc[i]["habitaciones"]-1<train_ref["habitaciones"]) & (train_ref["habitaciones"]<train_ref.loc[i]["habitaciones"]+1)].index
    rango = train_ref.loc[rango].loc[(train_ref.loc[i]["puntaje_por_fecha"]-0.3<train_ref["puntaje_por_fecha"]) & (train_ref["puntaje_por_fecha"]<train_ref.loc[i]["puntaje_por_fecha"]+0.3)].index
    if len(rango)<=1:
        rango = train_ref.loc[(train_ref.loc[i]["metroscubiertos"]-25<train_ref["metroscubiertos"]) & (train_ref["metroscubiertos"]<train_ref.loc[i]["metroscubiertos"]+25)].index
        rango = train_ref.loc[rango].loc[(train_ref.loc[i]["habitaciones"]-1<train_ref["habitaciones"]) & (train_ref["habitaciones"]<train_ref.loc[i]["habitaciones"]+1)].index
    if len(rango)<=1:
        rango = train_ref.loc[(train_ref.loc[i]["metroscubiertos"]-25<train_ref["metroscubiertos"]) & (train_ref["metroscubiertos"]<train_ref.loc[i]["metroscubiertos"]+25)].index
    if 1<len(rango)<7:
        knn = KNeighborsRegressor(n_neighbors=len(rango)-1, metric="manhattan", weights="distance", algorithm="brute")
    knn.fit(train.loc[rango].drop(i), precio.loc[rango].drop(i))
    id_predic.append(i)
    predic.append(knn.predict(train.loc[[i]]))
    if len(rango)<7:
        knn = KNeighborsRegressor(n_neighbors=7, metric="manhattan", weights="distance", algorithm="brute")

In [173]:
real = []
for x in id_predic:
    real.append(precio[x])
    
mean_absolute_error(real, predic)

940102.3234967493